In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.neighbors import NearestNeighbors
import folium


In [4]:
df=pd.read_csv('/content/Dataset .csv')

In [5]:
print(df)

      Restaurant ID           Restaurant Name  Country Code              City  \
0           6317637          Le Petit Souffle           162       Makati City   
1           6304287          Izakaya Kikufuji           162       Makati City   
2           6300002    Heat - Edsa Shangri-La           162  Mandaluyong City   
3           6318506                      Ooma           162  Mandaluyong City   
4           6314302               Sambo Kojin           162  Mandaluyong City   
...             ...                       ...           ...               ...   
9546        5915730               Naml۱ Gurme           208         ��stanbul   
9547        5908749              Ceviz A��ac۱           208         ��stanbul   
9548        5915807                     Huqqa           208         ��stanbul   
9549        5916112               A���k Kahve           208         ��stanbul   
9550        5927402  Walter's Coffee Roastery           208         ��stanbul   

                           

In [6]:
print(df.head())

   Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong City   
3      SM 

In [7]:
print(df.tail())

      Restaurant ID           Restaurant Name  Country Code       City  \
9546        5915730               Naml۱ Gurme           208  ��stanbul   
9547        5908749              Ceviz A��ac۱           208  ��stanbul   
9548        5915807                     Huqqa           208  ��stanbul   
9549        5916112               A���k Kahve           208  ��stanbul   
9550        5927402  Walter's Coffee Roastery           208  ��stanbul   

                                                Address     Locality  \
9546  Kemanke�� Karamustafa Pa��a Mahallesi, R۱ht۱m ...     Karak�_y   
9547  Ko��uyolu Mahallesi, Muhittin ��st�_nda�� Cadd...    Ko��uyolu   
9548  Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...  Kuru�_e��me   
9549  Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...  Kuru�_e��me   
9550  Cafea��a Mahallesi, Bademalt۱ Sokak, No 21/B, ...         Moda   

            Locality Verbose  Longitude   Latitude  \
9546     Karak�_y, ��stanbul  28.977392  41.022793   
9547    Ko��uy

In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [14]:
print(df.dropna())

      Restaurant ID           Restaurant Name  Country Code              City  \
0           6317637          Le Petit Souffle           162       Makati City   
1           6304287          Izakaya Kikufuji           162       Makati City   
2           6300002    Heat - Edsa Shangri-La           162  Mandaluyong City   
3           6318506                      Ooma           162  Mandaluyong City   
4           6314302               Sambo Kojin           162  Mandaluyong City   
...             ...                       ...           ...               ...   
9546        5915730               Naml۱ Gurme           208         ��stanbul   
9547        5908749              Ceviz A��ac۱           208         ��stanbul   
9548        5915807                     Huqqa           208         ��stanbul   
9549        5916112               A���k Kahve           208         ��stanbul   
9550        5927402  Walter's Coffee Roastery           208         ��stanbul   

                           

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# Handling missing values
df.dropna(subset=['Cuisines', 'Price range'], inplace=True)

# Convert 'Price range' to numerical if it's categorical
df['Price range'] = df['Price range'].astype(float)

# Vectorizing the 'Cuisines' column using TF-IDF
vectorizer = TfidfVectorizer()
cuisine_matrix = vectorizer.fit_transform(df['Cuisines'])

# Combine cuisine vectors with 'Price range'
X = hstack((cuisine_matrix, df[['Price range']].values)).tocsr()  # Convert to compressed sparse row format

# Model training
nn = NearestNeighbors(n_neighbors=5, metric='cosine')
nn.fit(X)

# Recommendation function
def recommend_restaurant(index):
    distances, indices = nn.kneighbors(X[index].toarray())  # Convert sparse row to dense array
    print("Recommended Restaurants:")
    for i in indices[0]:
        print(df.iloc[i]['Restaurant Name'])  # Replace with actual column name

recommend_restaurant(10)  # Example index


Recommended Restaurants:
Silantro Fil-Mex
Silantro Fil-Mex
Locavore
Guevarra's
Balay Dako


In [15]:
print(X)

  (0, 45)	0.3949328312995939
  (0, 57)	0.7256384522187065
  (0, 76)	0.5634508811097259
  (0, 150)	3.0
  (1, 76)	1.0
  (1, 150)	3.0
  (2, 10)	0.4471179676485894
  (2, 53)	0.7381348081610657
  (2, 69)	0.17171338550659074
  (2, 124)	0.4751389704335845
  (2, 150)	4.0
  (3, 76)	0.6690605668710697
  (3, 136)	0.7432078833396232
  (3, 150)	4.0
  (4, 76)	0.5961071302858968
  (4, 82)	0.8029049067120668
  (4, 150)	4.0
  (5, 35)	1.0
  (5, 150)	3.0
  (6, 10)	0.6737242242807269
  (6, 51)	0.7389828615180007
  (6, 150)	4.0
  (7, 10)	0.4062974671822443
  (7, 51)	0.44565247041012473
  (7, 53)	0.6707453618383769
  :	:
  (9533, 150)	3.0
  (9534, 28)	1.0
  (9534, 150)	2.0
  (9535, 1)	0.9320872231909288
  (9535, 45)	0.3622339138791721
  (9535, 150)	2.0
  (9536, 26)	0.4551840289506141
  (9536, 75)	0.8903973830758302
  (9536, 150)	3.0
  (9537, 144)	1.0
  (9537, 150)	3.0
  (9538, 28)	0.2377604261434737
  (9538, 42)	0.5527402926774335
  (9538, 110)	0.5647778982090831
  (9538, 149)	0.5647778982090831
  (9538, 15

In [16]:
print(nn)

NearestNeighbors(metric='cosine')


In [20]:
print(recommend_restaurant(10))

Recommended Restaurants:
Silantro Fil-Mex
Silantro Fil-Mex
Locavore
Guevarra's
Balay Dako
None
